In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import random
import math
import copy
import cv2

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.xception import preprocess_input, decode_predictions
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing import image_dataset_from_directory
#from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation
from tensorflow.keras.layers import RandomFlip, RandomRotation

from keras import backend as K
from sklearn.metrics import classification_report, confusion_matrix


2025-07-15 17:25:47.615722: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752600347.835451      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752600347.903615      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
! conda install -y gdown

/bin/bash: line 1: conda: command not found


In [3]:
import gdown

# a file
url = "https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/bwh3zbpkpv-1.zip"
gdown.download(url)

Downloading...
From: https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/bwh3zbpkpv-1.zip
To: /kaggle/working/bwh3zbpkpv-1.zip
100%|██████████| 8.44G/8.44G [04:11<00:00, 33.6MB/s] 


'bwh3zbpkpv-1.zip'

In [4]:
import zipfile
import os

zip_path = "/kaggle/working/bwh3zbpkpv-1.zip"
extract_path = "/kaggle/working/bwh3zbpkpv"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Unzipped to:", extract_path)


Unzipped to: /kaggle/working/bwh3zbpkpv


In [5]:
# List the top-level files and folders
for root, dirs, files in os.walk(extract_path):
    print("Current folder:", root)
    print("Subfolders:", dirs)
    print("Files:", files)
    break  # Remove this 'break' to see everything recursively


Current folder: /kaggle/working/bwh3zbpkpv
Subfolders: ['Dataset for Crop Pest and Disease Detection']
Files: []


In [6]:
import os

inner_folder = os.path.join(extract_path, "Dataset for Crop Pest and Disease Detection")

for root, dirs, files in os.walk(inner_folder):
    print("Current folder:", root)
    print("Subfolders:", dirs)
    print("Files:", files)
    break  # remove break if you want to go deeper


Current folder: /kaggle/working/bwh3zbpkpv/Dataset for Crop Pest and Disease Detection
Subfolders: ['CCMT Dataset-Augmented', 'Raw Data']
Files: []


In [7]:
ccmt_path = os.path.join(inner_folder, "CCMT Dataset-Augmented")

for root, dirs, files in os.walk(ccmt_path):
    print("Current folder:", root)
    print("Subfolders:", dirs)
    print("Files:", files)
    break


Current folder: /kaggle/working/bwh3zbpkpv/Dataset for Crop Pest and Disease Detection/CCMT Dataset-Augmented
Subfolders: ['Tomato', 'Maize', 'Cassava', 'Cashew']
Files: []


In [8]:
train_dir = os.path.join(
    inner_folder,
    "CCMT Dataset-Augmented",
    "Tomato",
    "train_set"
)

test_dir = os.path.join(
    inner_folder,
    "CCMT Dataset-Augmented",
    "Tomato",
    "test_set"
)

# validation_dir = os.path.join(
#     inner_folder,
#     "Raw Data",
#     "CCMT Dataset",
#     "Cashew",
# )
validation_dir = os.path.join(
    inner_folder,
    "CCMT Dataset-Augmented",
    "Tomato",
    "test_set"
)

# define train forms

In [10]:
import os

train_data = []

for class_name in os.listdir(train_dir):
    class_path = os.path.join(train_dir, class_name)
    if os.path.isdir(class_path):  # ✅ only process directories
        for file_name in os.listdir(class_path):
            file_path = os.path.join(class_path, file_name)
            train_data.append((file_path, class_name))

In [11]:
train_df = pd.DataFrame(train_data, columns=['File_Path', 'Class_Name'])
train_df.head()

,File_Path,Class_Name
0,/kaggle/working/bwh3zbpkpv/Dataset for Crop Pe...,verticulium wilt
1,/kaggle/working/bwh3zbpkpv/Dataset for Crop Pe...,verticulium wilt
2,/kaggle/working/bwh3zbpkpv/Dataset for Crop Pe...,verticulium wilt
3,/kaggle/working/bwh3zbpkpv/Dataset for Crop Pe...,verticulium wilt
4,/kaggle/working/bwh3zbpkpv/Dataset for Crop Pe...,verticulium wilt


In [12]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
train_df['Class_ID'] = encoder.fit_transform(train_df['Class_Name'])
train_df.head()

,File_Path,Class_Name,Class_ID
0,/kaggle/working/bwh3zbpkpv/Dataset for Crop Pe...,verticulium wilt,4
1,/kaggle/working/bwh3zbpkpv/Dataset for Crop Pe...,verticulium wilt,4
2,/kaggle/working/bwh3zbpkpv/Dataset for Crop Pe...,verticulium wilt,4
3,/kaggle/working/bwh3zbpkpv/Dataset for Crop Pe...,verticulium wilt,4
4,/kaggle/working/bwh3zbpkpv/Dataset for Crop Pe...,verticulium wilt,4


In [13]:
#train_df['Class_Name'].value_counts()

In [14]:
train_df['Class_ID'].value_counts()

Class_ID
3    9373
1    5200
4    3100
2    2050
0    2000
Name: count, dtype: int64

#take a look at the various diseases

In [15]:

#name is the title
def plotimage(desired_class: str,name):
    desired_class_df = train_df[train_df['Class_Name'] == desired_class]

    num_images_to_plot = 4

    fig, axes = plt.subplots(1, num_images_to_plot, figsize=(15, 5))

    for i, (index, row) in enumerate(desired_class_df.head(num_images_to_plot).iterrows()):
        image_path = row['File_Path']
        image = load_img(image_path)

        axes[i].imshow(image)
        axes[i].set_title(f"Image {i+1}: {name}")
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

In [16]:
#plotimage('healthy','HEALTHY')

In [17]:
#plotimage('red rust','RED RUST')

In [18]:
#plotimage('leaf miner','LEAF MINER')

In [19]:
#plotimage('anthracnose','ANTHRACNOSE')

In [20]:
#plotimage('gumosis','GUMOSIS')

# xception model

In [21]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
loss = 'categorical_crossentropy'
metrics = ['accuracy']

batch_size = 32

I0000 00:00:1752600671.110228      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [22]:
model1 = Xception(include_top=False,input_shape=(299, 299, 3), weights='imagenet')

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [23]:
input_shape= (299, 299)

In [24]:

datagen_train = ImageDataGenerator(rescale=1./255,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  vertical_flip=False)


datagen_test = ImageDataGenerator(rescale=1./255)
datagen_val = ImageDataGenerator(rescale=1./255)


generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=True)

generator_test = datagen_test.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)

generator_val = datagen_val.flow_from_directory(directory=validation_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)


Found 21723 images belonging to 5 classes.
Found 5445 images belonging to 5 classes.
Found 5445 images belonging to 5 classes.


In [25]:
math.ceil(generator_train.samples)

21723

In [26]:
next(generator_train)[1]

array([[0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

# Xception deep network

In [27]:
conv_model = Model(inputs=model1.input, outputs=model1.output)

In [28]:

new_model = Sequential()
new_model.add(conv_model)
new_model.add(Flatten())
new_model.add(Dropout(0.5))
new_model.add(Dense(512, activation='relu'))
new_model.add(Dense(5, activation='softmax'))

In [29]:
new_model.compile(optimizer= opt, loss=loss, metrics=metrics)

In [30]:

num_iters = 30000
num_batches_train = generator_train.n // batch_size

epochs = int(num_iters / num_batches_train)
epochs = 10
print("Epoch: ",epochs)
desired_train_accuracy = 0.99

#steps_per_epoch = generator_train.n // batch_size
#steps_val = generator_val.n // batch_size

steps_per_epoch = math.ceil(generator_train.n / batch_size)
steps_val = math.ceil(generator_val.n / batch_size)

print("Steps_per_epoch: ",steps_per_epoch)
print("Steps_val: ",steps_val)

Epoch:  10
Steps_per_epoch:  679
Steps_val:  171


# fit xception model

In [31]:

Checkpoint = ModelCheckpoint("xception_cassava.keras", monitor="val_accuracy", save_best_only=True, mode="max")
#EarlyStop = EarlyStopping(monitor="val_accuracy", baseline=desired_train_accuracy, patience=10, restore_best_weights=True, mode="auto")
EarlyStop = EarlyStopping(monitor="val_accuracy", patience=10, restore_best_weights=True,mode="auto")
history = new_model.fit(generator_train,
                        epochs=epochs,
                        callbacks=[Checkpoint, EarlyStop],
                        steps_per_epoch=steps_per_epoch,
                        validation_data=generator_val,
                        validation_steps=steps_val)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


I0000 00:00:1752600716.269104      97 service.cc:148] XLA service 0x78c21c003fb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752600716.270589      97 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1752600719.886850      97 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1752600736.324608      97 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752600736.563833      97 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752600737.911685      97 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752600738.1540

559/679 ━━━━━━━━━━━━━━━━━━━━ 1:33 776ms/step - accuracy: 0.5436 - loss: 1.2557

E0000 00:00:1752601191.996221      95 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752601192.236711      95 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752601192.877412      95 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752601193.142799      95 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752601197.319879      95 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

679/679 ━━━━━━━━━━━━━━━━━━━━ 673s 883ms/step - accuracy: 0.5622 - loss: 1.1922 - val_accuracy: 0.8009 - val_loss: 0.5243
Epoch 2/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 557s 820ms/step - accuracy: 0.8041 - loss: 0.5145 - val_accuracy: 0.8871 - val_loss: 0.2984
Epoch 3/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 522s 768ms/step - accuracy: 0.8829 - loss: 0.3140 - val_accuracy: 0.9309 - val_loss: 0.2041
Epoch 4/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 512s 753ms/step - accuracy: 0.9193 - loss: 0.2230 - val_accuracy: 0.9497 - val_loss: 0.1570
Epoch 5/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 503s 740ms/step - accuracy: 0.9366 - loss: 0.1682 - val_accuracy: 0.9309 - val_loss: 0.1944
Epoch 6/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 521s 767ms/step - accuracy: 0.9520 - loss: 0.1347 - val_accuracy: 0.9633 - val_loss: 0.1232
Epoch 7/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 540s 795ms/step - accuracy: 0.9576 - loss: 0.1225 - val_accuracy: 0.9579 - val_loss: 0.1444
Epoch 8/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 544s 800ms/step - accuracy: 0.9635 - loss: 0.10

In [32]:
# save exception model

In [33]:
# Delete the big zip file (if already extracted)
import os
os.remove('/kaggle/working/bwh3zbpkpv-1.zip')

In [34]:
import os

print(os.listdir('/kaggle/working'))


['xception_cassava.keras', '.virtual_documents', 'bwh3zbpkpv']


In [35]:
new_model.save('/kaggle/working/my_model.h5')

In [36]:
import os

print(os.listdir('/kaggle/working'))


['my_model.h5', 'xception_cassava.keras', '.virtual_documents', 'bwh3zbpkpv']


In [37]:
from IPython.display import FileLink

FileLink('/kaggle/working/my_model.h5')

/kaggle/working/my_model.h5

In [38]:
import shutil

shutil.make_archive('/kaggle/working/my_model', 'zip', '/kaggle/working', 'my_model.h5')


'/kaggle/working/my_model.zip'

In [39]:
import shutil

shutil.move("xception_cassava.keras", "/kaggle/working/xception_cassava.keras")

'/kaggle/working/xception_cassava.keras'

In [40]:


# Now try again:
import shutil
shutil.make_archive('/kaggle/working/my_model', 'zip', '/kaggle/working', 'my_model.h5')


'/kaggle/working/my_model.zip'

In [41]:
'''
def plot_training(history):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(new_model.history.history['accuracy'], label='Train Accuracy')
    plt.plot(new_model.history.history['val_accuracy'], label='Validation Accuracy')
    plt.axhline(0.99, color='red', linestyle='--', alpha=0.5, label='99% Target')
    plt.title('Accuracy Progress')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(new_model.history.history['loss'], label='Train Loss')
    plt.plot(new_model.history.history['val_loss'], label='Validation Loss')
    plt.title('Loss Progress')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Call this after each epoch
class PlotCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        plot_training(history)
        '''

"\ndef plot_training(history):\n    plt.figure(figsize=(12, 4))\n\n    plt.subplot(1, 2, 1)\n    plt.plot(new_model.history.history['accuracy'], label='Train Accuracy')\n    plt.plot(new_model.history.history['val_accuracy'], label='Validation Accuracy')\n    plt.axhline(0.99, color='red', linestyle='--', alpha=0.5, label='99% Target')\n    plt.title('Accuracy Progress')\n    plt.legend()\n\n    plt.subplot(1, 2, 2)\n    plt.plot(new_model.history.history['loss'], label='Train Loss')\n    plt.plot(new_model.history.history['val_loss'], label='Validation Loss')\n    plt.title('Loss Progress')\n    plt.legend()\n\n    plt.tight_layout()\n    plt.show()\n\n# Call this after each epoch\nclass PlotCallback(Callback):\n    def on_epoch_end(self, epoch, logs=None):\n        plot_training(history)\n        "

In [42]:
from tensorflow.keras.models import load_model

# Load your saved model
model = load_model('/kaggle/working/my_model.h5')

In [43]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.xception import preprocess_input

def predict_image_class(img_path):
    # Load and resize image
    img = image.load_img(img_path, target_size=(299, 299))

    # Convert to array and preprocess
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Make prediction
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]
    confidence = np.max(predictions)

    return predicted_class, confidence

In [44]:
# Get class names from your validation generator
class_names = list(generator_val.class_indices.keys())
class_mapping = {i: name for i, name in enumerate(class_names)}

print("Class Mapping:")
for idx, name in class_mapping.items():
    print(f"{idx}: {name}")

Class Mapping:
0: healthy
1: leaf blight
2: leaf curl
3: septoria leaf spot
4: verticulium wilt


In [45]:
import os
import random
from glob import glob

# Get all image paths from raw data directory
all_images = []
for root, dirs, files in os.walk(validation_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            all_images.append(os.path.join(root, file))

# Select random images
num_samples = 8
sample_images = random.sample(all_images, num_samples)

In [46]:
'''
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 10))
for i, img_path in enumerate(sample_images):
    # Get true class from directory structure
    true_class = img_path.split('/')[-2]

    # Predict
    pred_class_idx, confidence = predict_image_class(img_path)
    pred_class = class_mapping[pred_class_idx]

    # Plot
    img = image.load_img(img_path)
    plt.subplot(2, 4, i+1)
    plt.imshow(img)
    plt.title(f"True: {true_class}\nPred: {pred_class}\nConf: {confidence:.2f}")
    plt.axis('off')

    # Highlight incorrect predictions in red
    if true_class != pred_class:
        plt.gca().set_title(f"True: {true_class}\nPred: {pred_class}\nConf: {confidence:.2f}",
                            color='red')

plt.tight_layout()
plt.show()
'''

'\nimport matplotlib.pyplot as plt\n\nplt.figure(figsize=(15, 10))\nfor i, img_path in enumerate(sample_images):\n    # Get true class from directory structure\n    true_class = img_path.split(\'/\')[-2]\n\n    # Predict\n    pred_class_idx, confidence = predict_image_class(img_path)\n    pred_class = class_mapping[pred_class_idx]\n\n    # Plot\n    img = image.load_img(img_path)\n    plt.subplot(2, 4, i+1)\n    plt.imshow(img)\n    plt.title(f"True: {true_class}\nPred: {pred_class}\nConf: {confidence:.2f}")\n    plt.axis(\'off\')\n\n    # Highlight incorrect predictions in red\n    if true_class != pred_class:\n        plt.gca().set_title(f"True: {true_class}\nPred: {pred_class}\nConf: {confidence:.2f}",\n                            color=\'red\')\n\nplt.tight_layout()\nplt.show()\n'

In [47]:
''''from sklearn.metrics import classification_report

# Get all validation images
all_val_images = []
true_labels = []
for root, dirs, files in os.walk(validation_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            all_val_images.append(os.path.join(root, file))
            true_labels.append(os.path.basename(root))

# Predict all
pred_labels = []
for img_path in all_val_images:
    pred_class_idx, _ = predict_image_class(img_path)
    pred_labels.append(class_mapping[pred_class_idx])

# Classification report
print(classification_report(true_labels, pred_labels, target_names=class_names))
'''

"'from sklearn.metrics import classification_report\n\n# Get all validation images\nall_val_images = []\ntrue_labels = []\nfor root, dirs, files in os.walk(validation_dir):\n    for file in files:\n        if file.lower().endswith(('.png', '.jpg', '.jpeg')):\n            all_val_images.append(os.path.join(root, file))\n            true_labels.append(os.path.basename(root))\n\n# Predict all\npred_labels = []\nfor img_path in all_val_images:\n    pred_class_idx, _ = predict_image_class(img_path)\n    pred_labels.append(class_mapping[pred_class_idx])\n\n# Classification report\nprint(classification_report(true_labels, pred_labels, target_names=class_names))\n"

In [48]:
import shutil

shutil.rmtree('/kaggle/working/bwh3zbpkpv')


In [51]:
from IPython.display import FileLink

FileLink('my_model.zip')  # Or any file you want to download


/kaggle/working/my_model.zip